Cómo comarar una descomposición con otra luego de un outliear. Asumiendo que se tiene una descomposición "buena"; DEC2020, podemos ajustar con los dos modelos DEC2024 y DEC2024, y comparar MSE entre ellas, y entre la tasa oficial (no ajustada). Se puede hacer esto para comparar todos los modelos pre y post pandemia, y podemos obtener una base de erores comparando modelos dentro de los mismos periodos.

In [432]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

Será necesario convertir los indices desde tuplas (Año, Trimeste Móvil) a Datetimeindex para indexar.

In [433]:
from sklearn.metrics import mean_squared_error

def compared_series(pre_serie:pd.Series, pos_serie:pd.Series, outlier_start, outlier_end):
    """

    Parametros:
    pre_serie: Serie temporal antes del outliear
    pos_serie: Serie temporal después del outliear

    Las series deben tener el mismo multi indice ej: (Año, Trimestre) y deben estar orderanas de forma descendente. 
    """
    mse = mean_squared_error(pre_serie, pos_serie)
    mse_pre = mean_squared_error(pre_serie.loc[:outlier_start], pos_serie.loc[:outlier_start])
    print("MSE Series {pre}: ", mse)
    print("MSE pre outlier:", mse_pre)
    return mse, mse_pre

def mse_seasonality(pre_serie:pd.Series, pos_serie:pd.Series, outlier_start, outlier_end, pre_serie_comp:pd.Series=None, pos_serie_comp:pd.Series=None):
    """
    Parametros:
    pre_serie: Serie temporal antes del outliear
    pos_serie: Serie temporal después del outliear
    pre_serie_comp: Serie temporal comparativa antes del outliear
    pos_serie_comp: Serie temporal comparativa después del outliear

    Las series deben tener el mismo multi indice ej: (2020, "Abr- Jun"), una sola columna y deben estar orderanas de forma descendente. 

    Returns:
    model_diff: MSE de las series pre y pos pandemia menos el MSE encontrado en dos series pre pandemia
    pre_mse_comp:
    pos_mse_comp:
    """
    if pre_serie_comp is None:
        pre_serie_comp = pre_serie
    if pos_serie_comp is None:
        pos_serie_comp = pos_serie

    pos_serie = pos_serie.fillna(pos_serie_comp) # Se reemplaza solo con la serie similar
    pos_serie_comp = pos_serie_comp.fillna(pos_serie) # Lo mismo para anular la contribución con lo que resta de la serie similar    
    pos_mse_comp = mean_squared_error(pos_serie.loc[outlier_end:], pos_serie_comp.loc[outlier_end:])

    pre_serie = pre_serie.fillna(pre_serie_comp).fillna(pos_serie) # Se reemplaza para con la serie similar luego con la serie a comparar para anular la contribución
    pre_serie_comp = pre_serie_comp.fillna(pre_serie).fillna(pos_serie) # Lo mismo para anular la contribución con lo que resta de la serie similar
    pre_mse_comp = mean_squared_error(pre_serie.loc[:outlier_start], pre_serie_comp.loc[:outlier_start])
    pre_mse = mean_squared_error(pre_serie.loc[:outlier_start], pos_serie.loc[:outlier_start])
    return pre_mse - pre_mse_comp, pre_mse_comp, pos_mse_comp

In [434]:
def date_to_index(data):
    df = data.copy()
    trim_date = dict(zip(df[("Trimestre",np.nan)].unique(), (np.arange(12,0,-1) - 7)%12+1001)) # Ej: {'Dic - Feb': 1001, 'Nov - Feb': 1012}.  
    trim_date = {k:str(v).replace("100","0").replace("101","1") for k,v in trim_date.items()} # Anteriormente se sumo 1000 para anteponer un 0 al número, luego de convertido a string.
    df[("Trimestre",np.nan)] = df[("Trimestre",np.nan)].apply(lambda trim: trim_date[trim])
    df[("Trimestre", np.nan)]
    df["Date"] = df[["Año", "Trimestre"]].agg(lambda row: "-".join(row.astype("string")) + "-01", axis=1)
    df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
    df.index = df["Date"]
    df.drop(columns=[("Año",np.nan), ("Trimestre",np.nan), ("Date","")], inplace=True)
    df.sort_index(inplace=True)
    
    return df

data = pd.read_excel(io="../data/ine/ajuste_estacional_historico.xlsx", sheet_name="tasa_as")
data = data.loc[3:172]
data.columns= pd.MultiIndex.from_arrays(data.iloc[0:2].to_numpy())
data = data.loc[6:].reset_index(drop=True)
data = date_to_index(data=data)
data

,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,...,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada
,NDE2019,NDE2019,DEF2020,DEF2020,EFM2020,EFM2020,FMA2020,FMA2020,MAM2020,MAM2020,...,EFM2024,EFM2024,FMA2024,FMA2024,MAM2024,MAM2024,AMJ2024,AMJ2024,MJJ2024,MJJ2024
Date,,,,,,,,,,,,,,,,,,,,,
2010-08-01,8.1,7.9,8.124569,7.926932,8.124569,7.925845,8.124569,7.925795,8.124569,7.926,...,8.124569,7.928278,8.124569,7.928196,8.124569,7.928154,8.124569,7.928121,8.124569,7.928104
2010-09-01,7.8,7.7,7.814697,7.704444,7.814697,7.704655,7.814697,7.706665,7.814697,7.706946,...,7.814697,7.702712,7.814697,7.701814,7.814697,7.702325,7.814697,7.701711,7.814697,7.696071
2010-10-01,7.2,7.4,7.22505,7.427053,7.22505,7.428384,7.22505,7.426959,7.22505,7.42707,...,7.22505,7.418691,7.22505,7.419761,7.22505,7.418367,7.22505,7.418054,7.22505,7.41353
2010-11-01,7.2,7.5,7.210129,7.531558,7.210129,7.531692,7.210129,7.533507,7.210129,7.533247,...,7.210129,7.528851,7.210129,7.528725,7.210129,7.52886,7.210129,7.528905,7.210129,7.529803
2010-12-01,7.4,7.6,7.441762,7.646923,7.441762,7.647194,7.441762,7.6478,7.441762,7.647753,...,7.441762,7.666947,7.441762,7.666545,7.441762,7.667077,7.441762,7.667357,7.441762,7.67283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.677778,8.455696,8.677778,8.448077,8.677778,8.437456,8.677778,8.443967,8.677778,8.446938
2024-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.523784,8.359039,8.523784,8.34056,8.523784,8.357063,8.523784,8.366389


In [435]:
data.loc[:,("Tasa ajustada", "NDE2019")].loc["2023-05-01":"2023-07-01"]

Date
2023-05-01    NaN
2023-06-01    NaN
2023-07-01    NaN
Name: (Tasa ajustada, NDE2019), dtype: object

In [436]:
diff, premse, posmse = mse_seasonality(
    pre_serie=data[("Tasa ajustada", "NDE2019")],
    pos_serie=data[("Tasa ajustada", "MJJ2024")],
    pre_serie_comp=data[("Tasa ajustada", "EFM2020")],
    pos_serie_comp=data[("Tasa ajustada", "MJJ2023")],
    outlier_start="2020-03-01", outlier_end="2022-03-01"
    )

C:\Users\aurib\AppData\Local\Temp\ipykernel_41576\278034726.py:38: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\aurib\AppData\Local\Temp\ipykernel_41576\278034726.py:39: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\aurib\AppData\Local\Temp\ipykernel_41576\278034726.py:42: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C

In [437]:
print("diferencia: ", diff)
print("MSE previo al outlier", premse)
print("MSE posterior al outlier", posmse)

diferencia:  0.00019251550376636509
MSE previo al outlier 0.001112485417431904
MSE posterior al outlier 0.0026937396441335027


In [438]:
pdata = data.copy()
pdata.columns = pd.Index(map(str.strip, pdata.columns.get_level_values(0) + " " + pdata.columns.get_level_values(1)))
fig = px.line(pdata, y=["Tasa oficial DEF2020",  "Tasa ajustada DEF2020", "Tasa oficial DEF2024", "Tasa ajustada DEF2024"], 
              title="Tasa de desocupación y sus ajustes estacionales según DEF2020 y DEF2024",
              markers=True)
fig

In [439]:
from statsmodels.tsa.x13 import x13_arima_analysis as x13
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from os import path

x13as_path = path.abspath("C:/Program Files/x13as")
x13o = x13(
    endog=data.loc[:,("Tasa oficial", "DEF2024")].dropna(),
    maxorder=(1,1),
    forecast_periods=24,
    x12path=x13as_path,
    outlier=False)

In [440]:
print(x13o.seasadj)

Date
2010-08-01    7.973291
2010-09-01    7.639164
2010-10-01    7.428068
2010-11-01    7.524985
2010-12-01    7.645171
                ...   
2023-09-01    8.873446
2023-10-01    9.001044
2023-11-01    8.983292
2023-12-01    8.833883
2024-01-01    8.724162
Name: seasadj, Length: 162, dtype: float64


In [441]:
mean_squared_error(x13o.seasadj, data.loc[:,("Tasa ajustada", "DEF2024")].dropna())

np.float64(0.008792990951350228)

In [442]:
pdata = data.copy()
pdata.columns = pd.Index(map(str.strip, pdata.columns.get_level_values(0) + " " + pdata.columns.get_level_values(1)))
pdata = pd.concat([pdata.loc[:,["Tasa ajustada DEF2024", "Tasa oficial DEF2024"]].dropna(), x13o.seasadj], axis=1)\
    .convert_dtypes('float64')\
        .rename({"seasadj":"Tasa ajustada DEF2024 SM", "Tasa ajustada DEF2024":"Tasa ajustada DEF2024 INE", "Tasa oficial DEF2024":"Tasa oficial DEF2024 INE"}, axis=1)
fig = px.line(pdata, y=["Tasa ajustada DEF2024 INE", "Tasa ajustada DEF2024 SM", "Tasa oficial DEF2024 INE"], 
              title="Tasa de desocupación y sus ajustes estacionales según DEF2020 y DEF2024",
              markers=True)
fig

In [443]:
pan = pd.read_csv("D:\FCFM\Primavera2024\ProyectoINE\proyectomds_ine\data\pandemia\defunciones_covid19_2020_2024.csv", sep=";")
pan["FECHA_DEF"] = pd.to_datetime(pan["FECHA_DEF"], format="%Y-%m-%d")
pan.sort_values(by="FECHA_DEF", inplace=True)
pan.index = pan["FECHA_DEF"]

In [444]:
countpan = pan.groupby(by=[pan.index.year, pan.index.month]).count()
countpan.index = pd.to_datetime(countpan.index.to_series().apply(lambda tuple: "-".join([str(t) for t in tuple])+"-01"))


In [456]:
fig = px.line(countpan, y="EDAD_CANT", 
              title="Defunciones Mensuales COVID-19",
              markers=True)
fig

In [508]:
def composed_adjustment(data:pd.DataFrame, model="AMJ2023"):
    pre_pan = data.loc[:,("Tasa oficial", "NDE2019")].copy().astype('float64').dropna() # Modelo antes de la pandemia según OMS
    pre_pan.rename(" ".join(pre_pan.name), inplace=True)
    pos_pan = data.loc[:,("Tasa oficial", model)].copy().astype('float64').dropna()
    pos_pan.rename(" ".join(pos_pan.name), inplace=True)
    justpos_pan = pos_pan.loc["2020-01-01":].copy() # Modelo terminada la alerta de pandemia según OMS
    sari = SARIMAX(
        endog=pre_pan,
        order=(4,1,4),
        seasonal_order=(2,1,2,12),
        freq='MS'
        )

    results = sari.fit()
    fore24 = results.forecast(steps=29) # Desde DEF2020 hasta AMJ2022 fecha donde plan paso a paso 
    fore = pd.concat([pre_pan, fore24, justpos_pan], axis=1)
    fig = px.line(fore, y=["Tasa oficial NDE2019",  "predicted_mean"], 
                title="Tasa de desocupación y sus ajustes estacionales según DEF2020 y DEF2024",
                markers=True)
    fig.show()
    fig = px.line(fore, y=["Tasa oficial NDE2019",  "predicted_mean", f"Tasa oficial {model}"], 
                title="Tasa de desocupación y sus ajustes estacionales según DEF2020 y DEF2024",
                markers=True)
    fig.show()
    compose = fore["Tasa oficial NDE2019"].fillna(fore["predicted_mean"]).fillna(fore[f"Tasa oficial {model}"]).rename(f"Serie compuesta (DEF2020, {model})")
    fore_new = pd.concat([compose, justpos_pan], axis=1)
    fig = px.line(fore_new, y=[f"Tasa oficial {model}", f"Serie compuesta (DEF2020, {model})"],
                title="Tasa de desocupación y sus ajustes estacionales según DEF2020 y DEF2024",
                markers=True)
    fig.show()

    x13comp = x13(
        endog=fore_new[f"Serie compuesta (DEF2020, {model})"].dropna(),
        maxorder=(1,1),
        x12path=x13as_path,
        outlier=False)
    x13real = x13(
        endog=pos_pan.dropna(),
        maxorder=(1,1),
        x12path=x13as_path,
        outlier=False)
    comp_adj = x13comp.seasadj.rename("Ajuste estacional compuesto")
    real_adj = x13real.seasadj.rename("Ajuste estacional real")
    fig = px.line(pd.concat([comp_adj, real_adj, pos_pan], axis=1),
                title="Tasa de desocupación y sus ajustes estacionales según DEF2020 y DEF2024",
                markers=True)
    fig.show()
    print("MSE para Tasa compuesta y Tasa real: ", mean_squared_error(comp_adj, real_adj))


In [509]:
composed_adjustment(data)

d:\FCFM\Primavera2024\ProyectoINE\proyectomds_ine\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

d:\FCFM\Primavera2024\ProyectoINE\proyectomds_ine\.venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning:

Non-invertible starting seasonal moving average Using zeros as starting parameters.

d:\FCFM\Primavera2024\ProyectoINE\proyectomds_ine\.venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



MSE para Tasa compuesta y Tasa real:  1.2712639507216144


In [510]:
composed_adjustment(data, model="MJJ2024")

d:\FCFM\Primavera2024\ProyectoINE\proyectomds_ine\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

d:\FCFM\Primavera2024\ProyectoINE\proyectomds_ine\.venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning:

Non-invertible starting seasonal moving average Using zeros as starting parameters.

d:\FCFM\Primavera2024\ProyectoINE\proyectomds_ine\.venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



MSE para Tasa compuesta y Tasa real:  1.1712319230619168
